# 코드설명(임시)
- GAN 중간값 pca하는 코드

In [1]:
import tqdm

def nop(it, *a, **k):
    return it

real_tqdm = tqdm.tqdm
tqdm.tqdm = nop

import glob
import pickle

import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

np.bool = np.bool_


/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  mirr = onp.mirr
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:160: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  npv = onp.npv
/compuworks/anaconda3/envs/xaise

# PCA for zattr[0]
- 모든 데이터 사용
- PCA 컴포넌트 제한 없음 (역연산시 완전복원)
- minmax 한번에 구함

In [12]:
attr_id = 0

emb_array = []

emb_file_list = glob.glob("./examples/z_embeds/VggFace2-crop/*.pkl")

for file_path in real_tqdm(emb_file_list):
    with open(file_path, "rb") as file:
        emb_array.append(pickle.load(file)[attr_id].reshape([-1]))
emb_array = np.array(emb_array)


pca = PCA(svd_solver="full")
pca.fit(emb_array)
with open(f"zattr{attr_id}PCA.pkl", "wb") as file:
    pickle.dump(pca, file)

pca_array = pca.transform(emb_array)

pca_minmax = dict({"min": np.amin(pca_array, axis=0), "max": np.amax(pca_array, axis=0)})

with open(f"zattr{attr_id}PCAMinMax.pkl", "wb") as file:
    pickle.dump(pca_minmax, file)

# PCA for zattr[1, 2, 3, 4]
- 모든 데이터 사용
- PCA 컴포넌트 1024개 사용

In [2]:
for attr_id in [1, 2, 3, 4]:
    emb_array = []

    emb_file_list = glob.glob("./examples/z_embeds/VggFace2-crop/*.pkl")

    for file_path in real_tqdm(emb_file_list):
            with open(file_path, "rb") as file:
                emb_array.append(pickle.load(file)[attr_id].reshape([-1]))
    emb_array = np.array(emb_array)

    pca = PCA(n_components=1024, svd_solver="auto")

    pca.fit(emb_array)
    print("PCA가 다루는 분산 비율:", np.sum(pca.explained_variance_ratio_))
    with open(f"zattr{attr_id}PCA.pkl", "wb") as file:
        pickle.dump(pca, file)

    max_pca_array = -np.inf
    min_pca_array = np.inf

    for i in real_tqdm(range(emb_array.shape[0])):
        tmp = emb_array[i:i+1]
        pca_array = pca.transform(tmp)
        max_pca_array = np.maximum(max_pca_array, pca_array)
        min_pca_array = np.minimum(min_pca_array, pca_array)

    pca_minmax = dict({"min": min_pca_array[0], "max": max_pca_array[0]})

    with open(f"zattr{attr_id}PCAMinMax.pkl", "wb") as file:
        pickle.dump(pca_minmax, file)
    
    del pca, emb_array, pca_array, max_pca_array, min_pca_array, pca_minmax, tmp

0.9977929721355753
0.9799507086672928
0.8972457271133294


# PCA for zattr[5~6]
- 데이터의 약 80% 사용
- PCA 컴포넌트 1024개

In [1]:
for attr_id in [5, 6]:
    np.random.seed(7777)
    emb_array = []

    emb_file_list = glob.glob("./examples/z_embeds/VggFace2-crop/*.pkl")

    for file_path in real_tqdm(emb_file_list):
        if np.random.random() < 0.8:
            with open(file_path, "rb") as file:
                emb_array.append(pickle.load(file)[attr_id].reshape([-1]))
    emb_array = np.array(emb_array)

    pca = PCA(n_components=1024, svd_solver="auto")

    pca.fit(emb_array)
    print(np.sum(pca.explained_variance_ratio_))
    with open(f"zattr{attr_id}PCA.pkl", "wb") as file:
        pickle.dump(pca, file)

    max_pca_array = -np.inf
    min_pca_array = np.inf

    for i in real_tqdm(range(emb_array.shape[0])):
        tmp = emb_array[i:i+1]
        pca_array = pca.transform(tmp)
        max_pca_array = np.maximum(max_pca_array, pca_array)
        min_pca_array = np.minimum(min_pca_array, pca_array)

    pca_minmax = dict({"min": min_pca_array[0], "max": max_pca_array[0]})

    with open(f"zattr{attr_id}PCAMinMax.pkl", "wb") as file:
        pickle.dump(pca_minmax, file)
    
    del pca, emb_array, pca_array, max_pca_array, min_pca_array, pca_minmax, tmp

0.8293578069785364
0.8965594179673989


# PCA for zattr[7]
- 데이터의 절반만 사용
- PCA 컴포넌트 512개로 제한

In [1]:
attr_id = 7

emb_array = []

emb_file_list = glob.glob("./examples/z_embeds/VggFace2-crop/*.pkl")

for file_path in real_tqdm(emb_file_list[::2]):
    with open(file_path, "rb") as file:
        emb_array.append(pickle.load(file)[attr_id].reshape([-1]))
emb_array = np.array(emb_array)

pca = PCA(n_components=512, svd_solver="auto")

pca.fit(emb_array)
print(np.sum(pca.explained_variance_ratio_))
with open(f"zattr{attr_id}PCA.pkl", "wb") as file:
    pickle.dump(pca, file)



max_pca_array = -np.inf
min_pca_array = np.inf

for i in real_tqdm(range(emb_array.shape[0])):
    tmp = emb_array[i:i+1]
    pca_array = pca.transform(tmp)
    max_pca_array = np.maximum(max_pca_array, pca_array)
    min_pca_array = np.minimum(min_pca_array, pca_array)

pca_minmax = dict({"min": min_pca_array[0], "max": max_pca_array[0]})

with open(f"zattr{attr_id}PCAMinMax.pkl", "wb") as file:
    pickle.dump(pca_minmax, file)

0.8869378560189847


: 